# Model training
Training the actual model that we'll deploy after getting its architecture from the previous step (nn_exploration).

In [1]:
from utils import *

In [2]:
import pandas as pd

# Data importation

dataframe = pd.read_csv('data/data.csv')

#Getting the columns
columns = dataframe.columns.values

#Cleaning the dataframe
df = dataframe.drop(columns = columns[0:2])


In [8]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split, TensorDataset

#Defining input features and target feature
input_cols = columns[2:-1]
targets = columns[-1]

#Getting the tensors
inputs, targets = dataframe_to_torch(df,input_cols, targets)

#Creating the dataset
dataset = TensorDataset(inputs, targets)

dataset[2000]

(tensor([-0.2542,  0.0905,  1.1568, -1.1100]), tensor(1.))

In [ ]:

#Test:15% training: 85%

test_size = round(0.15*len(dataset))


train_ds, test_ds = random_split(dataset , [len(dataset) - test_size, test_size])

batch_size = 128  #Change based on GPU capacity

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size*2, num_workers=4, pin_memory=True)


In [5]:
# Input and ouput values of the hidden layers (input the values from the cross-validation step)
architecture = (28,60,30)

# Model definition 
model = DeepNeuralNetwork(4, *architecture)

In [9]:
#Hyperparameters
epochs = 15
max_lr = 0.0001
grad_clip = 0.007
weight_decay = 1e-5
opt_func = torch.optim.Adam

#Training
history = []

history += fit(epochs, max_lr, model, train_loader, test_loader, weight_decay, grad_clip, opt_func)

ValueError: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])) is deprecated. Please ensure they have the same size.

In [ ]:
plot_losses(history)

In [ ]:
#This step creates a readable model for a variety of programing languages.
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('model.pt') # Save
#ONNX export
dummy_input = torch.randn() #cambiar luego
torch.onnx.export(model, dummy_input, 'model.onnx', verbose=True)